# Monte Carlo Scanning
## Importing Libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import sixtrackwrap as sx

from parameters import *

import time

## Computing

In [2]:
problem = {
    'num_vars': 3,
    'names': ['alpha', 'theta1', 'theta2'],
    'bounds': [[-1, 1],
               [0, np.pi * 2],
               [0, np.pi * 2]]
    }
param_values = saltelli.sample(problem, mc_max_samples // 8 + 8)

alpha = np.array([np.arccos(p[0])/2 for p in param_values])
theta1 = np.array([p[1] for p in param_values])
theta2 = np.array([p[2] for p in param_values])

DA_3 = {}
error_3 = {}
data_3 = {}

# Data generation
time_start = time.time()

engine = sx.radial_scanner(alpha, theta1, theta2, d_r, starting_step=starting_step)
engine.scan(max_turns, min_turns, batch_size=batch_size)

radiuses = engine.extract_DA(turn_sampling)

data_3 = radiuses

elapsed_time_engine = time.time() - time_start

# Computing DA
time_start = time.time()

for sample in mc_samples:
    average = np.average(np.power(radiuses[:sample], 4), axis=0)
    error = np.std(np.power(radiuses[:sample], 4), axis=0) / np.sqrt(sample)
    DA_3[sample] = np.power(average, 1/4)
    error_3[sample] = 0.25 * np.power(average, -3/4) * error
    
elapsed_time_processing = time.time() - time_start

Active radiuses: 10064 / 10064
Sample size per active radius: 1
Expected execution time for step: nan
Elapsed time for whole iteration: 23.68
Time per single iteration: 2.3531611469857074e-05
r: 10.0 . Turns to do: 100 . Min found: 100
Active radiuses: 10064 / 10064
Sample size per active radius: 1
Expected execution time for step: 23.68
Elapsed time for whole iteration: 23.63
Time per single iteration: 2.3475488105904316e-05
r: 11.0 . Turns to do: 100 . Min found: 100
Active radiuses: 10064 / 10064
Sample size per active radius: 1
Expected execution time for step: 23.63
Elapsed time for whole iteration: 23.62
Time per single iteration: 2.3470855953204424e-05
r: 12.0 . Turns to do: 100 . Min found: 100
Active radiuses: 10064 / 10064
Sample size per active radius: 1
Expected execution time for step: 23.62
Elapsed time for whole iteration: 23.64
Time per single iteration: 2.3485734134865112e-05
r: 13.0 . Turns to do: 100 . Min found: 100
Active radiuses: 10064 / 10064
Sample size per act

### Saving data

In [5]:
with open(savepath + "data/DA_3.pkl", 'wb') as f:
    pickle.dump(DA_3, f, protocol=4)
    
with open(savepath + "data/error_3.pkl", 'wb') as f:
    pickle.dump(error_3, f, protocol=4)
    
with open(savepath + "data/time_3.pkl", "wb") as f:
    pickle.dump((elapsed_time_engine, elapsed_time_processing), f, protocol=4)